# 全国新型コロナウイルスデータ



## jsonデータを読み込み

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'AppleGothic'


data = pd.read_csv('./COVID-19.csv')
data.head()

,通し,厚労省NO,無症状病原体保有者,国内,チャーター便,年代,性別,確定日,発症日,受診都道府県,...,更新日時,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10
0,1,1,NaN,A-1,NaN,30,男性,1/15/2020,1/3/2020,神奈川県,...,3/8/2020 20:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,NaN,A-2,NaN,40,男性,1/24/2020,1/14/2020,東京都,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,NaN,A-3,NaN,30,女性,1/25/2020,1/21/2020,東京都,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,NaN,A-4,NaN,40,男性,1/26/2020,1/23/2020,愛知県,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,NaN,A-5,NaN,40,男性,1/28/2020,1/22/2020,愛知県,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 50 columns):
通し                  470 non-null int64
厚労省NO               429 non-null object
無症状病原体保有者           45 non-null object
国内                  301 non-null object
チャーター便              12 non-null object
年代                  470 non-null object
性別                  470 non-null object
確定日                 470 non-null object
発症日                 380 non-null object
受診都道府県              470 non-null object
居住都道府県              470 non-null object
居住管内                68 non-null object
居住市区町村              275 non-null object
キー                  470 non-null object
発表                  470 non-null object
ステータス               29 non-null object
備考                  115 non-null object
ソース                 404 non-null object
ソース2                426 non-null object
ソース3                48 non-null object
人数                  468 non-null float64
累計                  470 non-null int64
前日比                 4

## 症状別累計人数

In [3]:
total = len(data)
d = (data["ステータス"].str.contains('死亡')).sum()
di = (data["ステータス"].str.contains('退院')).sum()
print(data["確定日"].iloc[-1],"現在データ")
print('全国の新型コロナウイルス感染者はトータル:',total,'死亡:',d,'退院:',di)
print('死亡確率：{}%'.format((d/total)*100),'退院率：{}%'.format((di/total)*100))

3/8/2020 現在データ
全国の新型コロナウイルス感染者はトータル: 470 死亡: 7 退院: 21
死亡確率：1.4893617021276597% 退院率：4.468085106382979%


## 年代別感染者数割合

In [4]:
x = data['年代'].value_counts(ascending=True)
label_index = data['年代'].value_counts(ascending=True).index
plt.figure(figsize=(10,10))
plt.pie(x, labels= label_index, startangle=90, labeldistance=1,rotatelabels=False,autopct='%.2f%%')
plt.legend()
plt.show()

## 性別感染者数割合

In [5]:
x = data['性別'].value_counts(ascending=True)
label_index = data['性別'].value_counts(ascending=True).index
plt.figure(figsize=(10,10))
plt.pie(x, labels= label_index, startangle=90, labeldistance=1,rotatelabels=False,autopct='%.2f%%')
plt.legend()
plt.show()

### 県別累計データ

In [6]:
grouped = data['人数'].groupby(data['居住都道府県'])
total_pref = grouped.sum()
total_pref = total_pref.sort_values(ascending=False) 
total_pref = pd.DataFrame(total_pref)
total_pref

,人数
居住都道府県,
北海道,99.0
愛知県,68.0
東京都,64.0
大阪府,45.0
神奈川県,40.0
千葉県,19.0
中華人民共和国,18.0
和歌山県,14.0
兵庫県,12.0


In [7]:
location_data = pd.read_csv('./location.csv')
location_data.head()

,都道府県名,緯度,経度
0,北海道,43.06417,141.34694
1,青森県,40.82444,140.74000
2,岩手県,39.70361,141.15250
3,宮城県,38.26889,140.87194
4,秋田県,39.71861,140.10250


### locationデータと感染者数をマージ

In [8]:
pref_data = pd.merge(total_pref, location_data, left_on='居住都道府県', right_on='都道府県名', how='outer')
pref_data['人数'] = pref_data['人数'].fillna(0)
pref_data = pref_data.dropna(subset=['都道府県名'])
pref_data

,人数,都道府県名,緯度,経度
0,99.0,北海道,43.06417,141.34694
1,68.0,愛知県,35.18028,136.90667
2,64.0,東京都,35.68944,139.69167
3,45.0,大阪府,34.68639,135.52000
4,40.0,神奈川県,35.44778,139.64250
5,19.0,千葉県,35.60472,140.12333
7,14.0,和歌山県,34.22611,135.16750
8,12.0,兵庫県,34.69139,135.18306
9,11.0,高知県,33.55972,133.53111
10,10.0,埼玉県,35.85694,139.64889


In [9]:
pref_data_dic = pref_data.to_dict(orient='index')
states = tuple(pref_data_dic.values())

In [192]:
import folium
from folium.plugins import HeatMap

map_ = folium.Map(location=[35.684479, 139.747977], zoom_start=5, tiles='Stamen Toner')


# 円の大きさをわかりやすくするための重み
WEIGHT = 1

for state in states:
    folium.CircleMarker(
            [state['緯度'], state['経度']],
            radius=state['人数'] * WEIGHT,
            popup=state['都道府県名'],
            color='#3186cc',
            fill_color='#3186cc',
    ).add_to(map_)


map_.save('index.html')
map_
